In [258]:
#Test de la bibliothèque sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
# Chemin du dossier
dossier = "data-fr/Data_CAC40"
fichiers = os.listdir(dossier)
list_nom = []
for i in fichiers:
    chaine = i
    chaine_sans_dataframe = chaine.replace("dataframe_", "")
    second = chaine_sans_dataframe.replace(".csv","")
    list_nom.append(second)

print(list_nom)

['SU.PA', 'ORA.PA', 'ACA.PA', 'STM.PA', 'CAP.PA', 'DSY.PA', 'MC.PA', 'MT.AS', 'RI.PA', 'ENGI.PA', 'KER.PA', 'BN.PA', 'LR.PA', 'BNP.PA', '^FCHI', 'AIR.PA', 'SAN.PA', 'SGO.PA', 'GLE.PA', 'OR.PA', 'ML.PA', 'CA.PA', 'SAF.PA', 'AI.PA']


In [259]:
class portefeuille:
    def __init__(self, noms_titres, quantites, time_achat) -> None:
        self.noms_titres = noms_titres
        self.quantites = quantites
        self.time = time_achat
        self.data = []
        for i in self.noms_titres:
            path = "data-fr/Data_CAC40/dataframe_"+i+".csv"
            df = pd.read_csv(path)
            self.data.append(df)
            print(i, "chargé")
        self.recuperer_prix()
        self.complete_actions_df()

    def __str__(self) -> str:
        return f'{self.actions}'
    
    def get_noms_titres(self):
        return self.noms_titres

    def recuperer_prix(self):
        self.prix = []
        for i in self.data:
            self.prix.append(i['Close'][self.time])
    def get_data(self):
        return self.data
    
    def complete_actions_df(self):
        self.actions_temp = {'Prix':self.prix,
                        'Quantite': self.quantites}
        self.actions = pd.DataFrame(self.actions_temp, index=self.noms_titres)
        self.actions['Valeur'] = self.actions.apply(lambda row: row['Prix'] * row['Quantite'], axis=1)

    def add_predic(self, t, mod):
            predict = []
            for i in mod.models:
                predict.append(i.predict(np.array([t]).reshape(1,1))[0])
            self.actions['Prix_prévu_pour_t='+str(t)] = predict
            self.actions['Valeur du portefeuil_prévu_à_t='+str(t)] = self.actions.apply(lambda row: row['Prix_prévu_pour_t='+str(t)] * row['Quantite'], axis=1)

In [274]:
mon_portefeuille = portefeuille(list_nom, np.ones(len(list_nom)), time_achat=0)

SU.PA chargé
ORA.PA chargé
ACA.PA chargé
STM.PA chargé
CAP.PA chargé
DSY.PA chargé
MC.PA chargé
MT.AS chargé
RI.PA chargé
ENGI.PA chargé
KER.PA chargé
BN.PA chargé
LR.PA chargé
BNP.PA chargé
^FCHI chargé
AIR.PA chargé
SAN.PA chargé
SGO.PA chargé
GLE.PA chargé
OR.PA chargé
ML.PA chargé
CA.PA chargé
SAF.PA chargé
AI.PA chargé


### Les models

In [261]:
class lr_models:
    
    def __init__(self, portefeuille, period, learning_rate) -> None:
        self.mon_portefeuille = portefeuille 
        self.noms_titres = portefeuille.get_noms_titres()
        self.m = period
        self.learning_rate = learning_rate

    def create_models(self):
        self.models = []
        for i in self.mon_portefeuille.get_data():
            self.models.append(self.get_model(i))


    def get_model(self, df_titre):
        df = df_titre.head(self.m)
        self.Y = df['Close']
        self.X = np.arange(self.m).reshape(self.m,1)
        model = LinearRegression()
        model.fit(self.X,self.Y)
        #predict = model.predict(self.X)
        #plt.plot(self.X, predict, c = 'r')
        #plt.scatter(self.X,self.Y)
        print(model.score(self.X,self.Y))
        return model
    
    def apped_prediction():
        pass

In [276]:
mon_portefeuille = portefeuille(list_nom, np.ones(len(list_nom)), time_achat=0)
mod = lr_models(mon_portefeuille, 60, 0.01)
mod.create_models()

SU.PA chargé
ORA.PA chargé
ACA.PA chargé
STM.PA chargé
CAP.PA chargé
DSY.PA chargé
MC.PA chargé
MT.AS chargé
RI.PA chargé
ENGI.PA chargé
KER.PA chargé
BN.PA chargé
LR.PA chargé
BNP.PA chargé
^FCHI chargé
AIR.PA chargé
SAN.PA chargé
SGO.PA chargé
GLE.PA chargé
OR.PA chargé
ML.PA chargé
CA.PA chargé
SAF.PA chargé
AI.PA chargé
0.300368320679147
0.1480057683172663
0.03806981649911523
0.416719820599894
0.2745180734372309
0.45211357733127877
0.5011319592121676
0.009529542534768232
0.011590587054905788
0.2721032840642319
0.25683912576612367
0.21540872520306908
0.7780203919109118
0.01282323838499888
0.006090209561461446
0.36608100119088405
0.0056332285115442415
0.07133642265170914
0.1459129922103145
0.018513647535946753
0.05148977817269984
0.0978624535383894
0.7531121635452355
0.5162245191660817


In [277]:
mod.models[1].predict(np.array([60]).reshape(1,1))

array([7.55758013])

In [282]:
mon_portefeuille.add_predic(200,mod)

In [283]:
mon_portefeuille.actions.head(100)

,Prix,Quantite,Valeur,Prix_prévu_pour_t=60,Valeur du portefeuil_prévu_à_t=60,Prix_prévu_pour_t=200,Valeur du portefeuil_prévu_à_t=200
SU.PA,29.081612,1.0,29.081612,29.512416,29.512416,35.981348,35.981348
ORA.PA,7.673621,1.0,7.673621,7.557580,7.557580,8.179542,8.179542
ACA.PA,7.259819,1.0,7.259819,6.467827,6.467827,5.584103,5.584103
STM.PA,3.978039,1.0,3.978039,4.190367,4.190367,5.427284,5.427284
CAP.PA,25.837217,1.0,25.837217,27.623526,27.623526,33.407472,33.407472
DSY.PA,3.705438,1.0,3.705438,4.019801,4.019801,4.755512,4.755512
MC.PA,60.678326,1.0,60.678326,66.157933,66.157933,82.253282,82.253282
MT.AS,53.427956,1.0,53.427956,48.006894,48.006894,45.508875,45.508875
RI.PA,47.699757,1.0,47.699757,46.106830,46.106830,47.270980,47.270980
ENGI.PA,12.153281,1.0,12.153281,10.840577,10.840577,8.946812,8.946812


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [280]:
print(mon_portefeuille.actions)

                Prix  Quantite       Valeur  Prix_prévu_pour_t=60  \
SU.PA      29.081612       1.0    29.081612             29.512416   
ORA.PA      7.673621       1.0     7.673621              7.557580   
ACA.PA      7.259819       1.0     7.259819              6.467827   
STM.PA      3.978039       1.0     3.978039              4.190367   
CAP.PA     25.837217       1.0    25.837217             27.623526   
DSY.PA      3.705438       1.0     3.705438              4.019801   
MC.PA      60.678326       1.0    60.678326             66.157933   
MT.AS      53.427956       1.0    53.427956             48.006894   
RI.PA      47.699757       1.0    47.699757             46.106830   
ENGI.PA    12.153281       1.0    12.153281             10.840577   
KER.PA     57.814728       1.0    57.814728             61.284601   
BN.PA      28.888603       1.0    28.888603             29.140534   
LR.PA      15.265436       1.0    15.265436             18.191796   
BNP.PA     29.751303       1.0    